In [1]:
# fix imports
import os
import sys

module_path = os.path.abspath(os.path.join(".."))
if module_path not in sys.path:
    sys.path.append(module_path)

In [ ]:
import torch
from notebooks.experiment_robust import load_robust_experiment
from notebooks.experiment_torch import load_torchvision_experiment

model, dl_train, dl_eval = load_robust_experiment("Standard", "cifar10")
# model, dl_train, dl_eval = load_torchvision_experiment("vgg16")

In [ ]:
model

In [ ]:
from ulib.pert_module import PertModule
from ulib.attacks.spm.adversarial_attack import SPM_UAP
from ulib.attacks.spm.model_feature_extractor import ModelFeatureExtracter
from ulib import utils

device = utils.extract_device(model)

attacked_layer = model.block1.layer[3].conv1
mfe = ModelFeatureExtracter(model=model, layer=attacked_layer)

x, y = next(iter(dl_train))
output_shape = mfe.extract_layer_output(x.to(device)).shape[1:]

attack = SPM_UAP(
    input_shape=dl_train.data.shape[1:],
    output_shape=output_shape,
    device=device,
    verbose=True,
    p=float("inf"),
    q=10,
)

attack.fit(mfe, img_iter=dl_train)

In [ ]:
from ulib import eval

eps = 8 / 255
pert = attack.get_perturbation(eps=eps)
pert_model = PertModule(model, data_shape=dl_train.data.shape[1:], eps=eps)
pert_model.set_pert(pert)

eval.full_analysis(pert_model, dl_eval)